In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os.path
import pandas as pd
import seaborn as sns
from MyML.helper.plotting import save_fig

In [4]:
sns.set_style("whitegrid")
fig_width = 8
fig_height = 6

In [5]:
results_path = '/home/chiroptera/QCThesis/9thdimacs_results.csv'
dataset_metadata = '/home/chiroptera/QCThesis/9thdimacs_metadata.csv'

In [6]:
print '--- resulss ---'
res_raw = pd.read_csv(results_path)
for col in res_raw.columns:
    print col 

print '--- meta ---'
meta = pd.read_csv(dataset_metadata)
for col in meta.columns:
    print col 

--- resulss ---
dataset
param
mean
std
max
min
--- meta ---
dataset
E
V
MB


In [7]:
#meta = meta.applymap(lambda x:x.replace(',','.')) # replace , to . for float
#meta[['E','V','MB']] = meta[['E','V','MB']].astype(np.float)
meta = meta.set_index('dataset')

res = res_raw[np.logical_not(res_raw['mean'].isnull())] # ignore rows with null values
res = res.applymap(lambda x:x.replace(',','.')) # replace , to . for float
res[['mean','std','max','min']] = res[['mean','std','max','min']].astype(np.float)
res = res.set_index(['dataset','param'])

res['n_edges']=0
res['n_vertices']=0
res['mem']=0
res['edges_per_vertex'] = 0
for dataset in res.index.get_level_values('dataset').unique():
    res.loc[[dataset],'n_edges'] = meta.loc[dataset, 'E']
    res.loc[[dataset],'n_vertices'] = meta.loc[dataset,'V']
    res.loc[[dataset],'mem'] = meta.loc[dataset, 'MB']
res.loc[:,'edges_per_vertex'] = res.loc[:,'n_edges'] / res.loc[:,'n_vertices']

In [8]:
speedups = res.query('param=="SpeedUp"').loc[:,['n_vertices','n_edges','mean','edges_per_vertex','mem']]
speedups = speedups.reset_index()
del speedups['param']
speedups = speedups.set_index('dataset')

In [28]:
viz_speedup = speedups.copy()
#viz_speedup['cpu time'] = res.query('param in ("CPU")').loc[:,'mean'].values
#viz_speedup['gpu time'] = res.query('param in ("GPU")').loc[:,'mean'].values
idx_slow = viz_speedup.loc[:,'mean'] < 1
viz_speedup.loc[idx_slow,'mean'] = -1 / viz_speedup.loc[idx_slow,'mean']
print viz_speedup.sort('n_edges').to_latex(index_names=False)
viz_speedup.sort('n_edges')

\begin{tabular}{lrrrrr}
\toprule
{} &  n\_vertices &   n\_edges &      mean &  edges\_per\_vertex &         mem \\
\midrule
NY                           &      264347 &    730100 & -1.293761 &          2.761900 &    7.587030 \\
BAY                          &      321271 &    794830 & -1.254579 &          2.474017 &    8.515170 \\
COL                          &      435667 &   1042400 & -1.004995 &          2.392653 &   11.276800 \\
FLA                          &     1070377 &   2687902 &  1.389240 &          2.511173 &   28.673400 \\
NW                           &     1207946 &   2820774 &  1.451060 &          2.335182 &   30.736700 \\
NE                           &     1524454 &   3868020 &  1.559920 &          2.537315 &   41.141300 \\
CAL                          &     1890816 &   4630444 &  1.584020 &          2.448913 &   49.753300 \\
LKS                          &     2758120 &   6794808 &  1.699390 &          2.463565 &   72.883100 \\
E                            &     3598624 &

,n_vertices,n_edges,mean,edges_per_vertex,mem
dataset,,,,,
NY,264347,730100,-1.293761,2.761900,7.587030
BAY,321271,794830,-1.254579,2.474017,8.515170
COL,435667,1042400,-1.004995,2.392653,11.276800
FLA,1070377,2687902,1.389240,2.511173,28.673400
NW,1207946,2820774,1.451060,2.335182,30.736700
NE,1524454,3868020,1.559920,2.537315,41.141300
CAL,1890816,4630444,1.584020,2.448913,49.753300
LKS,2758120,6794808,1.699390,2.463565,72.883100
E,3598624,8708058,1.803500,2.419830,93.892500


In [10]:
print speedups.sort('n_edges').to_latex(index_names=False)
speedups.sort('n_edges')

\begin{tabular}{lrrrrr}
\toprule
{} &  n\_vertices &   n\_edges &     mean &  edges\_per\_vertex &         mem \\
\midrule
NY                           &      264347 &    730100 &  0.77294 &          2.761900 &    7.587030 \\
BAY                          &      321271 &    794830 &  0.79708 &          2.474017 &    8.515170 \\
COL                          &      435667 &   1042400 &  0.99503 &          2.392653 &   11.276800 \\
FLA                          &     1070377 &   2687902 &  1.38924 &          2.511173 &   28.673400 \\
NW                           &     1207946 &   2820774 &  1.45106 &          2.335182 &   30.736700 \\
NE                           &     1524454 &   3868020 &  1.55992 &          2.537315 &   41.141300 \\
CAL                          &     1890816 &   4630444 &  1.58402 &          2.448913 &   49.753300 \\
LKS                          &     2758120 &   6794808 &  1.69939 &          2.463565 &   72.883100 \\
E                            &     3598624 &   870805

,n_vertices,n_edges,mean,edges_per_vertex,mem
dataset,,,,,
NY,264347,730100,0.77294,2.761900,7.587030
BAY,321271,794830,0.79708,2.474017,8.515170
COL,435667,1042400,0.99503,2.392653,11.276800
FLA,1070377,2687902,1.38924,2.511173,28.673400
NW,1207946,2820774,1.45106,2.335182,30.736700
NE,1524454,3868020,1.55992,2.537315,41.141300
CAL,1890816,4630444,1.58402,2.448913,49.753300
LKS,2758120,6794808,1.69939,2.463565,72.883100
E,3598624,8708058,1.80350,2.419830,93.892500


In [11]:
print speedups.corr().to_latex()
speedups.corr()

\begin{tabular}{lrrrrr}
\toprule
{} &  n\_vertices &   n\_edges &      mean &  edges\_per\_vertex &       mem \\
\midrule
n\_vertices       &    1.000000 &  0.670382 &  0.692747 &         -0.217886 &  0.807464 \\
n\_edges          &    0.670382 &  1.000000 &  0.081862 &          0.578078 &  0.979033 \\
mean             &    0.692747 &  0.081862 &  1.000000 &         -0.653624 &  0.255672 \\
edges\_per\_vertex &   -0.217886 &  0.578078 & -0.653624 &          1.000000 &  0.399788 \\
mem              &    0.807464 &  0.979033 &  0.255672 &          0.399788 &  1.000000 \\
\bottomrule
\end{tabular}



,n_vertices,n_edges,mean,edges_per_vertex,mem
n_vertices,1.000000,0.670382,0.692747,-0.217886,0.807464
n_edges,0.670382,1.000000,0.081862,0.578078,0.979033
mean,0.692747,0.081862,1.000000,-0.653624,0.255672
edges_per_vertex,-0.217886,0.578078,-0.653624,1.000000,0.399788
mem,0.807464,0.979033,0.255672,0.399788,1.000000


In [12]:
res.query('param in("GPU","CPU","SpeedUp")')

mean      std       max       min  \
dataset                      param                                            
BAY                          CPU       0.05210  0.00010   0.05222   0.05196   
                             GPU       0.06540  0.00170   0.07018   0.06459   
                             SpeedUp   0.79708  0.02001   0.80849   0.74092   
CAL                          CPU       0.32844  0.00108   0.32964   0.32646   
                             GPU       0.20735  0.00081   0.20843   0.20614   
                             SpeedUp   1.58402  0.00665   1.59419   1.57183   
COL                          CPU       0.07045  0.00017   0.07071   0.07026   
                             GPU       0.07082  0.00085   0.07309   0.07015   
                             SpeedUp   0.99503  0.01087   1.00778   0.96748   
CTR                          CPU       3.76001  0.02664   3.83490   3.74697   
                             GPU       1.80073  0.00124   1.80244   1.79910   
                             SpeedUp   2.08805  0.01507   2.13035   2.07979   
E                            CPU       0.64647  0.00045   0.64724   0.64562   
                             GPU       0.35846  0.00112   0.36149   0.35767   
                             SpeedUp   1.80350  0.00595   1.80958   1.78782   
FLA                          CPU       0.17948  0.00047   0.18015   0.17842   
                             GPU       0.12920  0.00082   0.13021   0.12731   
                             SpeedUp   1.38924  0.00986   1.40913   1.37438   
LKS                          CPU       0.49772  0.00139   0.49952   0.49538   
                             GPU       0.29291  0.00277   0.29901   0.29013   
                             SpeedUp   1.69939  0.01485   1.71335   1.66549   
NE                           CPU       0.27123  0.00077   0.27210   0.26965   
                             GPU       0.17388  0.00054   0.17466   0.17309   
                             SpeedUp   1.55992  0.00626   1.56869   1.54832   
NW                           CPU       0.19497  0.00063   0.19657   0.19424   
                             GPU       0.13436  0.00044   0.13492   0.13365   
                             SpeedUp   1.45106  0.00499   1.45766   1.44357   
NY                           CPU       0.04789  0.00032   0.04871   0.04754   
                             GPU       0.06199  0.00162   0.06655   0.06104   
                             SpeedUp   0.77294  0.01966   0.79116   0.71899   
W                            CPU       1.12113  0.00129   1.12344   1.11872   
                             GPU       0.57929  0.00354   0.58447   0.57348   
                             SpeedUp   1.93543  0.01360   1.95691   1.91406   
Coassoc 50k sk=sqrt_2 th=30% CPU       2.23519  0.00269   2.23707   2.22775   
                             GPU      11.10442  0.01116  11.13339  11.09359   
                             SpeedUp   0.20129  0.00028   0.20161   0.20069   

                                       n_edges  n_vertices         mem  \
dataset                      param                                       
BAY                          CPU        794830      321271    8.515170   
                             GPU        794830      321271    8.515170   
                             SpeedUp    794830      321271    8.515170   
CAL                          CPU       4630444     1890816   49.753300   
                             GPU       4630444     1890816   49.753300   
                             SpeedUp   4630444     1890816   49.753300   
COL                          CPU       1042400      435667   11.276800   
                             GPU       1042400      435667   11.276800   
                             SpeedUp   1042400      435667   11.276800   
CTR                          CPU      34000000    14000000  365.819000   
                             GPU      34000000    14000000  365.819000   
                             SpeedUp  34000000    14000000  365.819000   
E  

In [13]:
print res.to_latex(index_names=False)

\begin{tabular}{llrrrrrrrr}
\toprule
    &     &      mean &      std &       max &       min &   n\_edges &  n\_vertices &         mem &  edges\_per\_vertex \\
\midrule
BAY & CPU &   0.05210 &  0.00010 &   0.05222 &   0.05196 &    794830 &      321271 &    8.515170 &          2.474017 \\
    & GPU &   0.06540 &  0.00170 &   0.07018 &   0.06459 &    794830 &      321271 &    8.515170 &          2.474017 \\
    & SpeedUp &   0.79708 &  0.02001 &   0.80849 &   0.74092 &    794830 &      321271 &    8.515170 &          2.474017 \\
    & Error &   0.00000 &  0.00000 &   0.00000 &   0.00000 &    794830 &      321271 &    8.515170 &          2.474017 \\
CAL & CPU &   0.32844 &  0.00108 &   0.32964 &   0.32646 &   4630444 &     1890816 &   49.753300 &          2.448913 \\
    & GPU &   0.20735 &  0.00081 &   0.20843 &   0.20614 &   4630444 &     1890816 &   49.753300 &          2.448913 \\
    & SpeedUp &   1.58402 &  0.00665 &   1.59419 &   1.57183 &   4630444 &     1890816 &   49.753300 &  